In [1]:
import pandas as pd

https://stackoverflow.com/questions/73180827/python-program-to-store-and-update-inventory

In [2]:
data = [
    {'item': 'book', 'price': 2, 'quantity': 20, 'store': 'a'}, 
    {'item': 'pen', 'price': 1, 'quantity': 100, 'store': 'a'}, 
    {'item': 'eraser', 'price': 3, 'quantity': 30, 'store': 'a'},
    {'item': 'book', 'price': 3, 'quantity': 30, 'store': 'b'}, 
    {'item': 'pen', 'price': 2, 'quantity': 20, 'store': 'b'}, 
    {'item': 'eraser', 'price': 1, 'quantity': 100, 'store': 'b'}, 
    {'item': 'book', 'price': 1, 'quantity': 100, 'store': 'c'},
    {'item': 'pen', 'price': 3, 'quantity': 30, 'store': 'c'}, 
    {'item': 'eraser', 'price': 2, 'quantity': 20, 'store': 'c'}
]

In [3]:
frame = pd.json_normalize(data)
frame

,item,price,quantity,store
0,book,2,20,a
1,pen,1,100,a
2,eraser,3,30,a
3,book,3,30,b
4,pen,2,20,b
5,eraser,1,100,b
6,book,1,100,c
7,pen,3,30,c
8,eraser,2,20,c


In [4]:
def operate(frame, item, quantity):
    # Filter item and sort by price:
    data = frame[frame["item"] == item].sort_values("price", ascending=True)
    # Compute cumulative quantity:
    data["cumsum"] = data["quantity"].cumsum()
    # Select store stocks to exhaust:
    taken = data[data["cumsum"] < quantity]
    # Select remaining stores:
    left = data.loc[data.index.difference(taken.index),:]
    # Quantity available on stock to exhaust
    amount = taken["quantity"].sum()
    # Exhaust stocks:
    taken["quantity"] = 0
    # Quantity to get from the remaining store:
    left.loc[left.index[0], "quantity"] -= quantity - amount
    # Assemble new stock state:
    order = pd.concat([taken, left])
    # Compute order quantity as well:
    order["order"] = data["quantity"] - order["quantity"]
    order["subtotal"] = order["order"]*order["price"]
    order = order[order["order"]>0]
    # Update data source:
    frame.loc[order.index, "quantity"] = order["quantity"]
    return order

In [5]:
order = operate(frame, "pen", 5)
print(order)

  item  price  quantity store  cumsum  order  subtotal
1  pen      1        95     a     100      5         5


In [6]:
print(frame)

     item  price  quantity store
0    book      2        20     a
1     pen      1        95     a
2  eraser      3        30     a
3    book      3        30     b
4     pen      2        20     b
5  eraser      1       100     b
6    book      1       100     c
7     pen      3        30     c
8  eraser      2        20     c


In [7]:
order = operate(frame, "pen", 120)
print(order)

  item  price  quantity store  cumsum  order  subtotal
1  pen      1         0     a      95     95        95
4  pen      2         0     b     115     20        40
7  pen      3        25     c     145      5        15


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [8]:
print(frame)

     item  price  quantity store
0    book      2        20     a
1     pen      1         0     a
2  eraser      3        30     a
3    book      3        30     b
4     pen      2         0     b
5  eraser      1       100     b
6    book      1       100     c
7     pen      3        25     c
8  eraser      2        20     c
